#Kaggle词袋遇上爆米花袋

In [2]:
!unzip word2vec-nlp-tutorial.zip && rm -rf word2vec-nlp-tutorial.zip
!unzip labeledTrainData.tsv.zip && rm -rf labeledTrainData.tsv.zip
!unzip testData.tsv.zip && rm -rf testData.tsv.zip
!unzip unlabeledTrainData.tsv.zip && rm -rf unlabeledTrainData.tsv.zip

Archive:  word2vec-nlp-tutorial.zip
  inflating: labeledTrainData.tsv.zip  
  inflating: sampleSubmission.csv    
  inflating: testData.tsv.zip        
  inflating: unlabeledTrainData.tsv.zip  
Archive:  labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    
Archive:  testData.tsv.zip
  inflating: testData.tsv            
Archive:  unlabeledTrainData.tsv.zip
  inflating: unlabeledTrainData.tsv  


In [3]:
import pandas as pd
import csv
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data = pd.read_csv('labeledTrainData.tsv', sep='\t').drop('id', axis=1)
test = pd.read_csv('testData.tsv', sep='\t').drop('id', axis=1)
test_id = pd.read_csv('testData.tsv', sep='\t')['id']
print(data.shape,test.shape)

data_train, data_eval = train_test_split(data, test_size=0.2, random_state=42)
print(data_train.shape, data_eval.shape)

(25000, 2) (25000, 1)
(20000, 2) (5000, 2)


In [4]:
#无需数据清洗，autogluon会自动处理
predictor = TabularPredictor(label='sentiment', eval_metric='accuracy')
model = predictor.fit(data_train,hyperparameters='multimodal', time_limit=900)

No path specified. Models will be saved in: "AutogluonModels/ag-20250615_043553"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          8
Memory Avail:       48.83 GB / 50.99 GB (95.8%)
Disk Space Avail:   183.20 GB / 225.83 GB (81.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitio

[1000]	valid_set's binary_error: 0.121


	0.882	 = Validation score   (accuracy)
	105.95s	 = Training   runtime
	0.24s	 = Validation runtime
Fitting model: MultiModalPredictor ... Training model for up to 47.55s of the 47.54s of remaining time.
		The total system num_gpus=0 is less than minimum num_gpus=1 to fit MultiModalPredictorModel. Consider using a machine with more GPUs.
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2169, in _train_and_save
    model = self._train_single(**model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2055, in _train_single
    model = model.fit(X=X, y=y, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, total_resources=total_resources, **model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [5]:
predict = model.predict(data_eval)
predict.head()

,sentiment
6868,0
24016,1
9668,0
13640,1
14018,0


In [6]:
model.evaluate(data_eval)

{'accuracy': 0.891,
 'balanced_accuracy': np.float64(0.8909161793185174),
 'mcc': np.float64(0.7821164287476686),
 'roc_auc': np.float64(0.9599208850303194),
 'f1': 0.8929062684220869,
 'precision': 0.8840466926070039,
 'recall': 0.9019452163556967}

In [7]:
model.leaderboard(data_eval, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,0.8916,0.8820,accuracy,1.140700,0.239209,105.954683,1.140700,0.239209,105.954683,1,True,6
1,WeightedEnsemble_L2,0.8910,0.8910,accuracy,1.820715,0.918108,442.687375,0.005092,0.000940,0.050033,2,True,7
2,LightGBMXT,0.8892,0.8795,accuracy,0.895751,0.180209,21.644805,0.895751,0.180209,21.644805,1,True,2
3,LightGBM,0.8886,0.8905,accuracy,0.959223,0.241473,37.720663,0.959223,0.241473,37.720663,1,True,1
4,CatBoost,0.8880,0.8790,accuracy,0.856400,0.675695,404.916679,0.856400,0.675695,404.916679,1,True,3
5,XGBoost,0.8824,0.8670,accuracy,1.021566,0.205530,156.621788,1.021566,0.205530,156.621788,1,True,4
6,NeuralNetTorch,0.6128,0.6245,accuracy,0.034108,0.021220,35.127019,0.034108,0.021220,35.127019,1,True,5


In [8]:
test_predict = model.predict(test)
test_predict.head()

,sentiment
0,1
1,0
2,1
3,1
4,1


In [9]:
#提交文件创建
submission = pd.DataFrame({'id':test_id,'sentiment':test_predict})
submission.to_csv('submission.csv',index=False)